In [1]:
import pandas as pd
df = pd.read_csv('/home/mane/code/minsolde/isolde-experiments/data/ebd6d29ca72afa.csv')
df

,implementation,solver,problem,num_txn,num_keys,num_values,num_sessions,synth_time_ms,check_time_ms,total_time_ms,initial_synth_clauses,total_synth_clauses,outcome,candidates,runId,endTime
0,all,GLUCOSE,RA_ax_b\tCC_ax_b,3,5,5,3,58,13,71,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:35
1,all,GLUCOSE,RA_ax_b\tCC_ax_b,3,5,5,3,35,9,45,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36
2,all,GLUCOSE,RA_ax_b\tCC_ax_b,3,5,5,3,35,7,43,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36
3,all,GLUCOSE,RA_ax_b\tCC_ax_b,4,5,5,3,22,7,30,9893,9893,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36
4,all,GLUCOSE,RA_ax_b\tCC_ax_b,4,5,5,3,23,7,30,9893,9893,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,no_learning,GLUCOSE,PC_ax_c\tPC_ax_b,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:03:14
2907,no_learning,GLUCOSE,SI_ax_b\tSI_ax_c,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:08:15
2908,no_learning,GLUCOSE,SI_ax_c\tSI_ax_b,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:13:15
2909,no_learning,GLUCOSE,Ser_ax_b\tSer_ax_c,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:18:15


In [2]:
import preprocessing as pre

df = pre.typify(df)
df

all problem sets are disjoint: True


,implementation,solver,problem,num_txn,num_keys,num_values,num_sessions,synth_time_ms,check_time_ms,total_time_ms,initial_synth_clauses,total_synth_clauses,outcome,candidates,runId,endTime,expected
0,all,glucose,RA_b|CC_b,3,5,5,3,58,13,71,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:35,SAT
1,all,glucose,RA_b|CC_b,3,5,5,3,35,9,45,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
2,all,glucose,RA_b|CC_b,3,5,5,3,35,7,43,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
3,all,glucose,RA_b|CC_b,4,5,5,3,22,7,30,9893,9893,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
4,all,glucose,RA_b|CC_b,4,5,5,3,23,7,30,9893,9893,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,no_learning,glucose,PC_c|PC_b,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:03:14,UNSAT
2907,no_learning,glucose,SI_b|SI_c,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:08:15,UNSAT
2908,no_learning,glucose,SI_c|SI_b,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:13:15,UNSAT
2909,no_learning,glucose,Ser_b|Ser_c,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:18:15,UNSAT


In [3]:
df[~(df['outcome'] == 'TIMEOUT') & (df['outcome'] != df['expected'])]

,implementation,solver,problem,num_txn,num_keys,num_values,num_sessions,synth_time_ms,check_time_ms,total_time_ms,initial_synth_clauses,total_synth_clauses,outcome,candidates,runId,endTime,expected
30,all,glucose,CC_b|PC_b,3,5,5,3,9,2,12,7534,8320,UNSAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
31,all,glucose,CC_b|PC_b,3,5,5,3,9,2,12,7534,8320,UNSAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
32,all,glucose,CC_b|PC_b,3,5,5,3,9,2,12,7534,8320,UNSAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
45,all,glucose,CC_c|PC_c,3,5,5,3,5,1,7,6490,6758,UNSAT,1,12/02/2026 07:42:35,12/02/2026 07:42:37,SAT
46,all,glucose,CC_c|PC_c,3,5,5,3,4,1,6,6490,6758,UNSAT,1,12/02/2026 07:42:35,12/02/2026 07:42:37,SAT
47,all,glucose,CC_c|PC_c,3,5,5,3,5,1,6,6490,6758,UNSAT,1,12/02/2026 07:42:35,12/02/2026 07:42:37,SAT
195,all,glucose,CC_b|PC_c,3,5,5,3,7,1,12,7527,8395,UNSAT,3,12/02/2026 07:42:35,12/02/2026 07:42:42,SAT
196,all,glucose,CC_b|PC_c,3,5,5,3,7,2,12,7527,8395,UNSAT,3,12/02/2026 07:42:35,12/02/2026 07:42:42,SAT
197,all,glucose,CC_b|PC_c,3,5,5,3,7,2,12,7527,8395,UNSAT,3,12/02/2026 07:42:35,12/02/2026 07:42:42,SAT
210,all,glucose,CC_c|PC_b,3,5,5,3,7,2,11,7803,9380,UNSAT,2,12/02/2026 07:42:35,12/02/2026 07:42:43,SAT


In [4]:
import domain as dom
df = df[~df['problem'].apply(lambda p: p.pos[0].name =='CC' and p.neg.name =='PC')]
df

,implementation,solver,problem,num_txn,num_keys,num_values,num_sessions,synth_time_ms,check_time_ms,total_time_ms,initial_synth_clauses,total_synth_clauses,outcome,candidates,runId,endTime,expected
0,all,glucose,RA_b|CC_b,3,5,5,3,58,13,71,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:35,SAT
1,all,glucose,RA_b|CC_b,3,5,5,3,35,9,45,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
2,all,glucose,RA_b|CC_b,3,5,5,3,35,7,43,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
3,all,glucose,RA_b|CC_b,4,5,5,3,22,7,30,9893,9893,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
4,all,glucose,RA_b|CC_b,4,5,5,3,23,7,30,9893,9893,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,no_learning,glucose,PC_c|PC_b,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:03:14,UNSAT
2907,no_learning,glucose,SI_b|SI_c,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:08:15,UNSAT
2908,no_learning,glucose,SI_c|SI_b,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:13:15,UNSAT
2909,no_learning,glucose,Ser_b|Ser_c,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:18:15,UNSAT


In [5]:
df = pre.trim(df, txn_num=(3,5), implementations=["all", "no_learning"])
df

,implementation,solver,problem,num_txn,num_keys,num_values,num_sessions,synth_time_ms,check_time_ms,total_time_ms,initial_synth_clauses,total_synth_clauses,outcome,candidates,runId,endTime,expected
0,all,glucose,RA_b|CC_b,3,5,5,3,58,13,71,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:35,SAT
1,all,glucose,RA_b|CC_b,3,5,5,3,35,9,45,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
2,all,glucose,RA_b|CC_b,3,5,5,3,35,7,43,7543,7543,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
3,all,glucose,RA_b|CC_b,4,5,5,3,22,7,30,9893,9893,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
4,all,glucose,RA_b|CC_b,4,5,5,3,23,7,30,9893,9893,SAT,1,12/02/2026 07:42:35,12/02/2026 07:42:36,SAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,no_learning,glucose,PC_c|PC_b,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:03:14,UNSAT
2907,no_learning,glucose,SI_b|SI_c,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:08:15,UNSAT
2908,no_learning,glucose,SI_c|SI_b,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:13:15,UNSAT
2909,no_learning,glucose,Ser_b|Ser_c,3,5,5,3,-1,-1,300000,-1,-1,TIMEOUT,-1,12/02/2026 07:42:35,13/02/2026 02:18:15,UNSAT


In [6]:
df = pre.merge_rows(df, check_num_measurements=False)
df

,implementation,solver,problem,num_txn,num_keys,num_values,num_sessions,candidates,initial_clauses,final_clauses,avg_time_ms,min_time_ms,max_time_ms,outcome,expected
0,all,glucose,CC_b|CC_c,3,5,5,3,3,7530,8398,11,11,11,UNSAT,UNSAT
1,all,glucose,CC_b|CC_c,4,5,5,3,36,10738,17330,316,313,321,UNSAT,UNSAT
2,all,glucose,CC_b|CC_c,5,5,5,3,483,17255,88805,6579,6379,6795,UNSAT,UNSAT
3,all,glucose,CC_b|RA_b,3,5,5,3,0,7522,7522,4,4,4,UNSAT,UNSAT
4,all,glucose,CC_b|RA_b,4,5,5,3,0,9855,9855,5,5,5,UNSAT,UNSAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,no_learning,glucose,Ser_b|Ser_c,3,5,5,3,-1,-1,-1,300000,300000,300000,TIMEOUT,UNSAT
190,no_learning,glucose,Ser_c|SI_c,3,5,5,3,-1,-1,-1,300000,300000,300000,TIMEOUT,UNSAT
191,no_learning,glucose,Ser_c|Ser_b,3,5,5,3,-1,-1,-1,300000,300000,300000,TIMEOUT,UNSAT
192,no_learning,glucose,TapCC_b|CC_b,3,5,5,3,-1,-1,-1,300000,300000,300000,TIMEOUT,UNSAT


In [27]:
# Group by your two features and get the index of the row with max 'b' in each group
dff = df[df['implementation'] == 'all'].copy()
dff['frameworks'] = dff['problem'].apply(lambda p: len(p.frameworks()))
result = dff.loc[dff.groupby(['frameworks', 'expected'])['avg_time_ms'].idxmax()]
result

,implementation,solver,problem,num_txn,num_keys,num_values,num_sessions,candidates,initial_clauses,final_clauses,avg_time_ms,min_time_ms,max_time_ms,outcome,expected,frameworks
74,all,glucose,SI_b US_b|Ser_b,5,5,5,3,7,28292,29663,54,54,55,SAT,SAT,1
50,all,glucose,RA_b|TapRA_b,5,5,5,3,0,41150,41150,110,109,112,UNSAT,UNSAT,1
77,all,glucose,SI_b US_b|Ser_c,5,5,5,3,13,95234,106112,102,101,103,SAT,SAT,2
113,all,glucose,Ser_b|Ser_c,5,5,5,3,103,15251,79713,9788,9498,10263,UNSAT,UNSAT,2


In [28]:
# Group by your two features and get the index of the row with max 'b' in each group
dff = df[df['implementation'] == 'no_learning'].copy()
dff = dff[dff['outcome'] != 'TIMEOUT']
dff['frameworks'] = dff['problem'].apply(lambda p: len(p.frameworks()))
result = dff.loc[dff.groupby(['frameworks', 'expected'])['avg_time_ms'].idxmax()]
result

,implementation,solver,problem,num_txn,num_keys,num_values,num_sessions,candidates,initial_clauses,final_clauses,avg_time_ms,min_time_ms,max_time_ms,outcome,expected,frameworks
182,no_learning,glucose,SI_c|Ser_c,5,5,5,3,535,10780,11314,82078,62619,101476,SAT,SAT,1
132,no_learning,glucose,CC_c|RA_c,5,5,5,3,0,0,0,347,7,1024,UNSAT,UNSAT,1
185,no_learning,glucose,SI_c US_c|Ser_b,3,5,5,3,1008,6035,7042,136267,101488,165325,SAT,SAT,2
156,no_learning,glucose,RA_b|RA_c,5,5,5,3,0,0,0,5143,5128,5159,UNSAT,UNSAT,2


In [4]:

df['avg_time_ms'] = df['avg_time_ms'].replace(300000, '-')

df_pivoted = df.pivot(
    index='problem',           # Rows
    columns='num_txn',         # New columns
    values='avg_time_ms'           # Values to fill
)

# Rename columns to be more descriptive
df_pivoted.columns = [f'time_ms_txn_{col}' for col in df_pivoted.columns]

# Reset index if you want 'problem' as a regular column
df_pivoted = df_pivoted.reset_index()


df_pivoted["sat_order"] = df_pivoted["problem"].apply(lambda p: "a" if sat(p) else "b")
df_pivoted["sat"] = df_pivoted["problem"].apply(lambda p: "Yes" if sat(p) else "No")
df_pivoted["frameworks"] = df_pivoted["problem"].apply(lambda p: rf'\({showframeworks(frameworks(p))}\)')
df_pivoted["length_fw"] = df_pivoted["frameworks"].apply(len)


# Create a mapping from value to position
order_map = {val: idx for idx, val in enumerate(all_problems_flatten)}

# Create temporary sort key
df_pivoted['sort_key'] = df_pivoted['problem'].map(order_map)

df_pivoted = df_pivoted.sort_values(by=['sat_order', 'length_fw', 'frameworks', 'sort_key']).reset_index(drop=True)
df_pivoted["problem_str"] = df_pivoted["problem"].apply(problem_as_latex)

# Drop the temporary column
df_pivoted = df_pivoted.drop('sort_key', axis=1)
df_pivoted = df_pivoted.drop('length_fw', axis=1)
df_pivoted = df_pivoted.drop('sat_order', axis=1)


df_pivoted

,problem,time_ms_txn_3,time_ms_txn_4,time_ms_txn_5,time_ms_txn_6,time_ms_txn_7,time_ms_txn_8,time_ms_txn_9,time_ms_txn_10,sat,frameworks,problem_str
0,RA_b\tCC_b,NaN,30,44,36,NaN,78,NaN,60,Yes,\(\mathcal{B}\),"$\{\textsc{RA}_{\mathcal{B}}, \, \overline{\te..."
1,CC_b\tPC_b,NaN,12,43,49,NaN,357,NaN,213,Yes,\(\mathcal{B}\),"$\{\textsc{CC}_{\mathcal{B}}, \, \overline{\te..."
2,PC_b\tSI_b,NaN,16,44,51,NaN,120,NaN,282,Yes,\(\mathcal{B}\),"$\{\textsc{PC}_{\mathcal{B}}, \, \overline{\te..."
3,SI_b\tSer_b,NaN,16,39,43,NaN,132,NaN,549,Yes,\(\mathcal{B}\),"$\{\textsc{SI}_{\mathcal{B}}, \, \overline{\te..."
4,SI_b UpdateSer_b\tSer_b,NaN,37,57,87,NaN,488,NaN,957,Yes,\(\mathcal{B}\),"$\{\textsc{SI}_{\mathcal{B}},\ \textsc{US}_{\m..."
5,PlumeRA_b\tRA_b,NaN,17,30,35,NaN,109,NaN,187,Yes,\(\mathcal{B}\),"$\{\textsc{TapRA}_{\mathcal{B}}, \, \overline{..."
6,RA_c\tCC_c,NaN,8,22,14,NaN,29,NaN,3022,Yes,\(\mathcal{C}\),"$\{\textsc{RA}_{\mathcal{C}}, \, \overline{\te..."
7,CC_c\tPC_c,NaN,7,27,8,NaN,22,NaN,24,Yes,\(\mathcal{C}\),"$\{\textsc{CC}_{\mathcal{C}}, \, \overline{\te..."
8,PC_c\tSI_c,NaN,6,10,8,NaN,14,NaN,22,Yes,\(\mathcal{C}\),"$\{\textsc{PC}_{\mathcal{C}}, \, \overline{\te..."
9,SI_c\tSer_c,NaN,9,16,11,NaN,28,NaN,1442,Yes,\(\mathcal{C}\),"$\{\textsc{SI}_{\mathcal{C}}, \, \overline{\te..."


In [5]:
select_problems = [
    ([("SI", "c"), ("UpdateSer", "c")], ("Ser", "c")), 
    ([("SI", "c"), ("UpdateSer", "b")], ("Ser", "c")), 
    ([("Ser", "b")], ("SI", "b")), 
    ([("SI", "c")], ("SI", "b")), 
]

for index, row in df_pivoted.iterrows():
    problem = parse(row['problem'])
    if problem in select_problems:
        print(f'{row['problem_str']} & \\textbf{{{row['sat']}}} & \\textbf{{{row['frameworks']}}} & \\textbf{{{row['time_ms_txn_4']}}} & \\textbf{{{row['time_ms_txn_5']}}} & \\textbf{{{row['time_ms_txn_6']}}} & \\textbf{{{row['time_ms_txn_8']}}} & \\textbf{{{row['time_ms_txn_10']}}} \\\\')
    else:
        print(f'{row['problem_str']} & {row['sat']} & {row['frameworks']} & {row['time_ms_txn_4']} & {row['time_ms_txn_5']} & {row['time_ms_txn_6']} & {row['time_ms_txn_8']} & {row['time_ms_txn_10']} \\\\')

$\{\textsc{RA}_{\mathcal{B}}, \, \overline{\textsc{CC}_{\mathcal{B}}}\}$ & Yes & \(\mathcal{B}\) & 30 & 44 & 36 & 78 & 60 \\
$\{\textsc{CC}_{\mathcal{B}}, \, \overline{\textsc{PC}_{\mathcal{B}}}\}$ & Yes & \(\mathcal{B}\) & 12 & 43 & 49 & 357 & 213 \\
$\{\textsc{PC}_{\mathcal{B}}, \, \overline{\textsc{SI}_{\mathcal{B}}}\}$ & Yes & \(\mathcal{B}\) & 16 & 44 & 51 & 120 & 282 \\
$\{\textsc{SI}_{\mathcal{B}}, \, \overline{\textsc{Ser}_{\mathcal{B}}}\}$ & Yes & \(\mathcal{B}\) & 16 & 39 & 43 & 132 & 549 \\
$\{\textsc{SI}_{\mathcal{B}},\ \textsc{US}_{\mathcal{B}}, \, \overline{\textsc{Ser}_{\mathcal{B}}}\}$ & Yes & \(\mathcal{B}\) & 37 & 57 & 87 & 488 & 957 \\
$\{\textsc{TapRA}_{\mathcal{B}}, \, \overline{\textsc{RA}_{\mathcal{B}}}\}$ & Yes & \(\mathcal{B}\) & 17 & 30 & 35 & 109 & 187 \\
$\{\textsc{RA}_{\mathcal{C}}, \, \overline{\textsc{CC}_{\mathcal{C}}}\}$ & Yes & \(\mathcal{C}\) & 8 & 22 & 14 & 29 & 3022 \\
$\{\textsc{CC}_{\mathcal{C}}, \, \overline{\textsc{PC}_{\mathcal{C}}}\}$ & Yes & 

In [6]:
# new table
for index, row in df_pivoted.iterrows():
    problem = row['problem']
    if problem in problems_new_table1:
        print(f'{row['problem_str']} & {row['sat']} & {row['frameworks']} & {row['time_ms_txn_5']} \\\\')

$\{\textsc{SI}_{\mathcal{B}},\ \textsc{US}_{\mathcal{B}}, \, \overline{\textsc{Ser}_{\mathcal{B}}}\}$ & Yes & \(\mathcal{B}\) & 57 \\
$\{\textsc{SI}_{\mathcal{B}},\ \textsc{US}_{\mathcal{C}}, \, \overline{\textsc{Ser}_{\mathcal{C}}}\}$ & Yes & \(\mathcal{B},\ \mathcal{C}\) & 170 \\
$\{\textsc{CC}_{\mathcal{B}}, \, \overline{\textsc{TapCC}_{\mathcal{B}}}\}$ & No & \(\mathcal{B}\) & 106 \\
$\{\textsc{RA}_{\mathcal{B}}, \, \overline{\textsc{RA}_{\mathcal{C}}}\}$ & No & \(\mathcal{B},\ \mathcal{C}\) & 22638 \\


In [6]:
for index, row in df_pivoted.iterrows():
    problem = parse(row['problem'])
    if problem in select_problems:
        print(f'{row['problem_str']} & {row['sat']} & {row['frameworks']} & {row['time_ms_txn_4']} & {row['time_ms_txn_6']} & {row['time_ms_txn_8']} & {row['time_ms_txn_10']} \\\\')

$\{\textsc{SI}_{\mathcal{C}},\ \textsc{US}_{\mathcal{C}}, \, \overline{\textsc{Ser}_{\mathcal{C}}}\}$ & Yes & \(\mathcal{C}\) & 14 & 74 & 285 & 5544 \\
$\{\textsc{SI}_{\mathcal{C}},\ \textsc{US}_{\mathcal{B}}, \, \overline{\textsc{Ser}_{\mathcal{C}}}\}$ & Yes & \(\mathcal{B},\ \mathcal{C}\) & 35 & 137 & 485 & 21765 \\
$\{\textsc{Ser}_{\mathcal{B}}, \, \overline{\textsc{SI}_{\mathcal{B}}}\}$ & No & \(\mathcal{B}\) & 26 & 209 & 2358 & 20845 \\
$\{\textsc{SI}_{\mathcal{C}}, \, \overline{\textsc{SI}_{\mathcal{B}}}\}$ & No & \(\mathcal{B},\ \mathcal{C}\) & 881 & 4762 & 40830 & 252558 \\


In [7]:
df = clean_rq2(pd.read_csv('/home/mane/vldb_measurements/data.csv'))
df

,implementation,solver,problem,num_txn,num_keys,num_values,num_sessions,candidates,avg_time_ms,min_time_ms,max_time_ms,sat,frameworks,frameworks_str,optimizations
196,default,glucose,SI_c\tSI_b,3,5,5,3,1,110,90,147,UNSAT,"[b, c]","\mathcal{B},\ \mathcal{C}","\(\{ S, \, T\}\)"
197,default,glucose,SI_c\tSI_b,4,5,5,3,3,881,871,887,UNSAT,"[b, c]","\mathcal{B},\ \mathcal{C}","\(\{ S, \, T\}\)"
198,default,glucose,SI_c\tSI_b,5,5,5,3,4,1078,1005,1142,UNSAT,"[b, c]","\mathcal{B},\ \mathcal{C}","\(\{ S, \, T\}\)"
199,default,glucose,SI_c\tSI_b,6,5,5,3,7,4762,4730,4820,UNSAT,"[b, c]","\mathcal{B},\ \mathcal{C}","\(\{ S, \, T\}\)"
200,default,glucose,SI_c\tSI_b,7,5,5,3,7,11099,10770,11376,UNSAT,"[b, c]","\mathcal{B},\ \mathcal{C}","\(\{ S, \, T\}\)"
201,default,glucose,SI_c\tSI_b,8,5,5,3,8,40830,40287,41249,UNSAT,"[b, c]","\mathcal{B},\ \mathcal{C}","\(\{ S, \, T\}\)"
202,default,glucose,SI_c\tSI_b,9,5,5,3,24,167987,131009,196895,UNSAT,"[b, c]","\mathcal{B},\ \mathcal{C}","\(\{ S, \, T\}\)"
203,default,glucose,SI_c\tSI_b,10,5,5,3,15,252558,212545,293496,UNSAT,"[b, c]","\mathcal{B},\ \mathcal{C}","\(\{ S, \, T\}\)"
240,default,glucose,Ser_b\tSI_b,3,5,5,3,0,26,19,33,UNSAT,[b],\mathcal{B},"\(\{ S, \, T\}\)"
241,default,glucose,Ser_b\tSI_b,4,5,5,3,0,26,26,27,UNSAT,[b],\mathcal{B},"\(\{ S, \, T\}\)"


In [8]:
df['avg_time_ms'] = df['avg_time_ms'].replace(300000, '-')

df_pivoted = df.pivot(
    index=['problem', 'implementation'],           # Rows
    columns='num_txn',         # New columns
    values='avg_time_ms'           # Values to fill
)

# Rename columns to be more descriptive
df_pivoted.columns = [f'time_ms_txn_{col}' for col in df_pivoted.columns]

cols = [c for c in df_pivoted.columns if c.startswith("time_ms_txn_")]

# Reset index if you want 'problem' as a regular column
df_pivoted = df_pivoted.reset_index()

# Create a mapping from value to position
order_map = {val: idx for idx, val in enumerate(["default", "without fixed order", "without smart search", "no optimizations"])}

# Create temporary sort key
df_pivoted['sort_key'] = df_pivoted['implementation'].map(order_map)

# Create a mapping from value to position
order_map_problems = {val: idx for idx, val in enumerate(all_problems_flatten)}

# Create temporary sort key
df_pivoted['sort_key_problem'] = df_pivoted['problem'].map(order_map_problems)

df_pivoted = df_pivoted.sort_values(by=['sort_key_problem', 'sort_key']).reset_index(drop=True)
df_pivoted["problem_str"] = df_pivoted["problem"].apply(problem_as_latex)

# Drop the temporary column
df_pivoted = df_pivoted.drop('sort_key', axis=1)
df_pivoted = df_pivoted.drop('sort_key_problem', axis=1)

df_pivoted

,problem,implementation,time_ms_txn_3,time_ms_txn_4,time_ms_txn_5,time_ms_txn_6,time_ms_txn_7,time_ms_txn_8,time_ms_txn_9,time_ms_txn_10,problem_str
0,Ser_b\tSI_b,default,26,26,97,209,1229,2358,6740,20845,"$\{\textsc{Ser}_{\mathcal{B}}, \, \overline{\t..."
1,Ser_b\tSI_b,without fixed order,30,537,5493,67021,NaN,-,NaN,NaN,"$\{\textsc{Ser}_{\mathcal{B}}, \, \overline{\t..."
2,Ser_b\tSI_b,without smart search,25,561,319,13244,NaN,10566,NaN,59329,"$\{\textsc{Ser}_{\mathcal{B}}, \, \overline{\t..."
3,Ser_b\tSI_b,no optimizations,50,249,4194,31653,NaN,-,NaN,NaN,"$\{\textsc{Ser}_{\mathcal{B}}, \, \overline{\t..."
4,SI_c\tSI_b,default,110,881,1078,4762,11099,40830,167987,252558,"$\{\textsc{SI}_{\mathcal{C}}, \, \overline{\te..."
5,SI_c\tSI_b,without fixed order,158,1169,8950,91258,NaN,-,NaN,NaN,"$\{\textsc{SI}_{\mathcal{C}}, \, \overline{\te..."
6,SI_c\tSI_b,without smart search,36,161,3554,3260,NaN,156015,NaN,229518,"$\{\textsc{SI}_{\mathcal{C}}, \, \overline{\te..."
7,SI_c\tSI_b,no optimizations,110,960,12117,77000,NaN,-,NaN,NaN,"$\{\textsc{SI}_{\mathcal{C}}, \, \overline{\te..."


In [9]:
for problem_name, group in df_pivoted.groupby('problem', sort=False):
    s = f"\\midrule\n\\multirow{{4}}{{*}}{{{problem_as_latex(problem_name)}}} "
    for _, row in group.iterrows():
        s += f'& {optimizations(row['implementation'])} & {row['time_ms_txn_4']} & {row['time_ms_txn_6']} & {row['time_ms_txn_8']} & {row['time_ms_txn_10']}\\\\\n'
    print(s,end='')

\midrule
\multirow{4}{*}{$\{\textsc{Ser}_{\mathcal{B}}, \, \overline{\textsc{SI}_{\mathcal{B}}}\}$} & \(\{ S, \, T\}\) & 26 & 209 & 2358 & 20845\\
& \(\{ S \}\) & 537 & 67021 & - & nan\\
& \(\{ T \}\) & 561 & 13244 & 10566 & 59329\\
& \(\emptyset\) & 249 & 31653 & - & nan\\
\midrule
\multirow{4}{*}{$\{\textsc{SI}_{\mathcal{C}}, \, \overline{\textsc{SI}_{\mathcal{B}}}\}$} & \(\{ S, \, T\}\) & 881 & 4762 & 40830 & 252558\\
& \(\{ S \}\) & 1169 & 91258 & - & nan\\
& \(\{ T \}\) & 161 & 3260 & 156015 & 229518\\
& \(\emptyset\) & 960 & 77000 & - & nan\\
